In [ ]:
import numpy as np
import scipy.stats as sps
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display
dtype = np.float64
from printmd import  *
float_formatter = "{:.4f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})

## Типовой расчет по предмету МСА. Часть 1
### Остромецкий Дмитрий Вариант 15
**Выборка 1**

In [ ]:
k = 15
sample_size = 20
def Y_l(i, j):
    i+=1
    j+=1
    return  i / (i + j)

def Y_b(j):
    j+=1
    return j * (k + 1) / k

Y_linear = np.fromfunction(Y_l, (4, 4), dtype=dtype)
Y_bias = np.fromfunction(Y_b, (4,), dtype=dtype)
Y = np.matmul(sps.norm.rvs(size=(sample_size, 4)), Y_linear) + Y_bias
pd.DataFrame(Y, columns=['y(1)', 'y(2)', 'y(4)', 'y(4)'])

**Выборка 2**

In [ ]:
def X_l(i, j):
    i+=1
    j+=1
    return  (i + 1)  / (i + j + 1)

def X_b(j):
    j+=1
    return j * (k + 3) / k

X_linear = np.fromfunction(Y_l, (4, 4), dtype=dtype)
X_bias = np.fromfunction(Y_b, (4,), dtype=dtype)
X = np.matmul(sps.norm.rvs(size=(sample_size, 4)), Y_linear) + Y_bias
pd.DataFrame(Y, columns=['x(1)', 'x(2)', 'x(4)', 'x(4)'])

1. **Найти векторы из оценок средних для обеих выборок**

In [ ]:
Y_m = Y.mean(axis=0)
X_m = X.mean(axis=0)
Y_D = np.cov(Y.T)
l, e = np.linalg.eig(Y_D)

display(mdVector("\\hat{\\mu}_X", X_m))
display(mdVector("\\hat{\\mu}_Y", Y_m))

2. **Для элементов первой выборки найти первую и вторую главные
компоненты**

In [ ]:
Y_D = np.cov(Y.T)
display(mdMatrix( "\\hat{D}_Y", Y_D))

In [ ]:
l, e = np.linalg.eig(Y_D)
plt.plot(range(len(l)), l / l.sum(), 'o-', linewidth=2, color='blue')
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.show()

In [ ]:
pc1 = np.matmul(Y - Y_m, e[0].T)
plt.hist(np.matmul(Y - Y_m, e[0]), bins=5, density=True)
plt.show()
print(np.sort(pc1))

In [ ]:
pc3 = np.matmul(Y - Y_m, e[2].T)
plt.scatter(pc1, pc3)
plt.xlabel("Principal component 1")
plt.ylabel("Principal component 3")
plt.show()

3. **Для второй выборки найти линейную комбинацию центрированных 2-ой, 3-ей, 4-ой компонент x(2), x(3), x(4), имеющую наибольший коэффициент корреляции с центрированной первой компонентой x(1).**

In [ ]:
X_center = X - X_m
X_std = np.power(np.cov(X.T).diagonal(), 1/2)

X_norm = X_center / X_std
# np.cov(X_norm.T)


In [ ]:
from sklearn.cross_decomposition import CCA
X_norm[ :,:1]
cca = CCA(n_components=1)
cca.fit(X_norm[ :,:1], X_norm[ :,1:])
X_c, Y_c = cca.transform(X_norm[ :,:1], X_norm[ :,1:])
display(mdVector("\\vec{\\lambda}", cca.x_rotations_.ravel()))
display(mdVector("\\vec{\\beta}", cca.y_rotations_.ravel()))
plt.scatter(X_c, Y_c)
plt.xlabel(r"$\vec{v}$")
plt.ylabel(r"$\vec{u}$")
plt.show()


4. **Объединить обе выборки в одну выборку, вставив вторую выборку под первой выборкой (x(1) под y(1) и т.д.). Получить выборку объема 40.**

In [ ]:
G = np.vstack((Y, X))
Z = np.ones((sample_size * 2, 2))
Z[sample_size:, 1] = 2

pd.DataFrame(G, columns=['g(1)', 'g(2)', 'g(4)', 'g(4)'])

5. **Оценить параметры линейной регрессии $E\vec{\gamma} = B\vec{z} = B\begin{pmatrix} 1 \\ z \end{pmatrix}$, где $B$ – матрица нагрузок порядка (4х2).**